# Importing Required Libraries


In [1]:
import pandas as pd

import numpy as np

import pickle

import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns

import sklearn

from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import RandomizedSearchCV

import imblearn

from imblearn.under_sampling import RandomUnderSampler

from sklearn.preprocessing import scale

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

import warnings

warnings.filterwarnings('ignore')

# Reading the Dataset

In [2]:
df = pd.read_csv('autos.csv', parse_dates=['dateCrawled', 'dateCreated', 'lastSeen'], encoding = 'latin1')

In [3]:
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31,0,60437,2016-04-06 10:17:21


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   dateCrawled          371528 non-null  datetime64[ns]
 1   name                 371528 non-null  object        
 2   seller               371528 non-null  object        
 3   offerType            371528 non-null  object        
 4   price                371528 non-null  int64         
 5   abtest               371528 non-null  object        
 6   vehicleType          333659 non-null  object        
 7   yearOfRegistration   371528 non-null  int64         
 8   gearbox              351319 non-null  object        
 9   powerPS              371528 non-null  int64         
 10  model                351044 non-null  object        
 11  kilometer            371528 non-null  int64         
 12  monthOfRegistration  371528 non-null  int64         
 13  fuelType      

In [5]:
df.shape

(371528, 20)

# Cleaning the Dataset

In [6]:
df.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [7]:
# Rearranging the Columns
df = df[['dateCrawled', 'name', 'seller', 'offerType', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen', 'price']]

In [8]:
# Droping the Unwanted Columns
df.drop(columns= ['seller', 'offerType', 'nrOfPictures'], inplace = True)

In [9]:
df.drop(columns= ['dateCrawled', 'dateCreated', 'lastSeen'], inplace = True)

# Missing Values

In [10]:
# Checking for Missing Values
df.isna().sum()

name                       0
abtest                     0
vehicleType            37869
yearOfRegistration         0
gearbox                20209
powerPS                    0
model                  20484
kilometer                  0
monthOfRegistration        0
fuelType               33386
brand                      0
notRepairedDamage      72060
postalCode                 0
price                      0
dtype: int64

In [11]:
# Removing Missing Values
df['vehicleType'].fillna(df['vehicleType'].mode()[0], inplace = True)
df['gearbox'].fillna(df['gearbox'].mode()[0], inplace = True)
df['model'].fillna(df['model'].mode()[0], inplace = True)
df['fuelType'].fillna(df['fuelType'].mode()[0], inplace = True)
df['notRepairedDamage'].fillna(df['notRepairedDamage'].mode()[0], inplace = True)

In [12]:
df.isna().sum()

name                   0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
postalCode             0
price                  0
dtype: int64

# Duplicate Values

In [13]:
# Checking for Duplicates
df.duplicated().sum()

4703

In [14]:
# Removing Duplicates
df = df.drop_duplicates()

In [15]:
df.duplicated().sum()

0

# Label Encoding

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366825 entries, 0 to 371527
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   name                 366825 non-null  object
 1   abtest               366825 non-null  object
 2   vehicleType          366825 non-null  object
 3   yearOfRegistration   366825 non-null  int64 
 4   gearbox              366825 non-null  object
 5   powerPS              366825 non-null  int64 
 6   model                366825 non-null  object
 7   kilometer            366825 non-null  int64 
 8   monthOfRegistration  366825 non-null  int64 
 9   fuelType             366825 non-null  object
 10  brand                366825 non-null  object
 11  notRepairedDamage    366825 non-null  object
 12  postalCode           366825 non-null  int64 
 13  price                366825 non-null  int64 
dtypes: int64(6), object(8)
memory usage: 42.0+ MB


In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['name'] = le.fit_transform(df['name'])
df['abtest'] = le.fit_transform(df['abtest'])
df['vehicleType'] = le.fit_transform(df['vehicleType'])
df['gearbox'] = le.fit_transform(df['gearbox'])
df['model'] = le.fit_transform(df['model'])
df['fuelType'] = le.fit_transform(df['fuelType'])
df['brand'] = le.fit_transform(df['brand'])
df['notRepairedDamage'] = df['notRepairedDamage'].replace({'nein' : 0, 'ja' : 1})

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366825 entries, 0 to 371527
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   name                 366825 non-null  int32
 1   abtest               366825 non-null  int32
 2   vehicleType          366825 non-null  int32
 3   yearOfRegistration   366825 non-null  int64
 4   gearbox              366825 non-null  int32
 5   powerPS              366825 non-null  int64
 6   model                366825 non-null  int32
 7   kilometer            366825 non-null  int64
 8   monthOfRegistration  366825 non-null  int64
 9   fuelType             366825 non-null  int32
 10  brand                366825 non-null  int32
 11  notRepairedDamage    366825 non-null  int64
 12  postalCode           366825 non-null  int64
 13  price                366825 non-null  int64
dtypes: int32(7), int64(7)
memory usage: 32.2 MB


# Identifying and Handling Outliers

In [19]:
df.skew()

name                     0.004977
abtest                  -0.073059
vehicleType             -0.875930
yearOfRegistration      71.721506
gearbox                 -1.449355
powerPS                 58.001367
model                    0.405910
kilometer               -1.557831
monthOfRegistration      0.080801
fuelType                 1.489454
brand                   -0.151812
notRepairedDamage        2.703224
postalCode               0.059804
price                  574.388798
dtype: float64

In [21]:
df.yearOfRegistration=df.yearOfRegistration.clip(lower=df.yearOfRegistration.quantile(0.05),upper=df.yearOfRegistration.quantile(0.95))

df.price=df.price.clip(lower=df.price.quantile(0.05),upper=df.price.quantile(0.95))

df.powerPS=df.powerPS.clip(lower=df.powerPS.quantile(0.05),upper=df.powerPS.quantile(0.95))

In [22]:
df.skew()

name                   0.004977
abtest                -0.073059
vehicleType           -0.875930
yearOfRegistration     0.158712
gearbox               -1.449355
powerPS                0.077325
model                  0.405910
kilometer             -1.557831
monthOfRegistration    0.080801
fuelType               1.489454
brand                 -0.151812
notRepairedDamage      2.703224
postalCode             0.059804
price                  1.416900
dtype: float64